## Fully Automate Data Cleaning with Python

In [1]:
import pandas as pd

### Step 1: Run Basic Data Quality Checks
 
We need to identify:
- Missing values in each column
- Duplicate rows
- Basic data characteristics

In [2]:
def check_data_quality(df):
    # Store initial data quality metrics
    quality_report = {
        'missing_values': df.isnull().sum().to_dict(),
        'duplicates': df.duplicated().sum(),
        'total_rows': len(df),
        'memory_usage': df.memory_usage().sum() / 1024**2  # in MB
    }
    return quality_report

# This gives us a baseline understanding of our data's quality and helps identify the specific cleaning tasks we'll need to perform.

### Step 2: Standardize Data Types

We ensure all fields have the right/expected data types. This includes:

- Converting string dates to datetime objects
- Identifying and converting numeric strings to actual numbers
- Ensuring categorical variables are properly encoded

In [4]:
def standardize_datatypes(df):
    for column in df.columns:
        # Try converting string dates to datetime
        if df[column].dtype == 'object':
            try:
                df[column] = pd.to_datetime(df[column])
                print(f"Converted {column} to datetime")
            except ValueError:
                # Try converting to numeric if datetime fails
                try:
                    df[column] = pd.to_numeric(df[column].str.replace('$', '').str.replace(',', ''))
                    print(f"Converted {column} to numeric")
                except:
                    pass
    return df

# This step prevents type-related errors in subsequent analysis.

### Step 3: Handle Missing Values
Rather than dropping data records with missing values, we can use imputation strategies:

- Using median imputation for numeric columns
- Applying mode imputation for categorical data
- Maintaining the statistical properties of the dataset while filling gaps

In [5]:
from sklearn.impute import SimpleImputer

def handle_missing_values(df):
    # Handle numeric columns
    numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
    if len(numeric_columns) > 0:
        num_imputer = SimpleImputer(strategy='median')
        df[numeric_columns] = num_imputer.fit_transform(df[numeric_columns])
    
    # Handle categorical columns
    categorical_columns = df.select_dtypes(include=['object']).columns
    if len(categorical_columns) > 0:
        cat_imputer = SimpleImputer(strategy='most_frequent')
        df[categorical_columns] = cat_imputer.fit_transform(df[categorical_columns])
    
    return df

### Step 4: Detect and Handle Outliers

Here's an approach using the Interquartile Range (IQR) method:

- Calculate Interquartile Range (IQR) for numeric columns
- Identify values beyond 1.5 * IQR from quartiles
- Apply capping to extreme values rather than removing them

This preserves data while managing extreme values.

In [6]:
def remove_outliers(df):
    numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
    outliers_removed = {}
    
    for column in numeric_columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Count outliers before removing
        outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)].shape[0]
        
        # Cap the values instead of removing them
        df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)
        
        if outliers > 0:
            outliers_removed[column] = outliers
            
    return df, outliers_removed

### Step 5: Validate the Results

After cleaning, we need to verify that our pipeline worked as expected:

- Confirm no remaining missing values
- Check for any remaining duplicates
- Validate data integrity and consistency
- Generate a comprehensive cleaning report

In [7]:
def validate_cleaning(df, original_shape, cleaning_report):
    validation_results = {
        'rows_remaining': len(df),
        'missing_values_remaining': df.isnull().sum().sum(),
        'duplicates_remaining': df.duplicated().sum(),
        'data_loss_percentage': (1 - len(df)/original_shape[0]) * 100
    }
    
    # Add validation results to the cleaning report
    cleaning_report['validation'] = validation_results
    return cleaning_report

In [8]:
## Put everything in complete pipeline
def automated_cleaning_pipeline(df):
    # Store original shape for reporting
    original_shape = df.shape
    
    # Initialize cleaning report
    cleaning_report = {}
    
    # Execute each step and collect metrics
    cleaning_report['initial_quality'] = check_data_quality(df)
    
    df = standardize_datatypes(df)
    df = handle_missing_values(df)
    df, outliers = remove_outliers(df)
    cleaning_report['outliers_removed'] = outliers
    
    # Validate and finalize report
    cleaning_report = validate_cleaning(df, original_shape, cleaning_report)
    
    return df, cleaning_report

In [9]:
## Test the pipeline
from sklearn.datasets import load_iris
import numpy as np

dataframe = load_iris()
dataframe = pd.DataFrame(data= np.c_[dataframe['data'], dataframe['target']],
                  columns= dataframe['feature_names'] + ['Species'])

automated_cleaning_pipeline(dataframe)

(     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
 0                  5.1               3.5                1.4               0.2   
 1                  4.9               3.0                1.4               0.2   
 2                  4.7               3.2                1.3               0.2   
 3                  4.6               3.1                1.5               0.2   
 4                  5.0               3.6                1.4               0.2   
 ..                 ...               ...                ...               ...   
 145                6.7               3.0                5.2               2.3   
 146                6.3               2.5                5.0               1.9   
 147                6.5               3.0                5.2               2.0   
 148                6.2               3.4                5.4               2.3   
 149                5.9               3.0                5.1               1.8   
 
      Species 